In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from matplotlib import pyplot as plt
import pickle
import import_ipynb
import sys
sys.path.append('./../')
import Portfolio_value as pv
import statsmodels.api as sm

In [2]:
df =  pd.read_csv('./../data/data.csv')

In [3]:
p_val = np.load('./../result/p_val/p_val_const_delaying.npy', allow_pickle=True)
w_val = np.load('./../result/w_val/w_val_const_delaying.npy', allow_pickle=True)

p_val = p_val.reshape(1)
p_val = p_val[0]

w_val = w_val.reshape(1)
w_val = w_val[0]

In [4]:
normal_year = np.load('./../result/year/normal_year.npy', allow_pickle=True).reshape(1)[0]
bear_year = np.load('./../result/year/bear_year.npy', allow_pickle=True).reshape(1)[0]
full_year = np.load('./../result/year/full_year.npy', allow_pickle=True).reshape(1)[0]

In [5]:
upper_years = [20, 25, 30]
withdraw_rates = [0.03, 0.051] # 

In [6]:
normal_perform = {}
bear_perform = {}
full_perform = {}
for upper_year in upper_years:
    normal_perform[upper_year] = {
    'dep':[],
    'max':[],
    'under':[],
    'inc':[],
    'beq':[],
    'total':[]
}
    bear_perform[upper_year] = {
    'dep':[],
    'max':[],
    'under':[],
    'inc':[],
    'beq':[],
    'total':[]
}
    
    full_perform[upper_year] = {
    'dep':[],
    'max':[],
    'under':[],
    'inc':[],
    'beq':[],
    'total':[]
}

In [7]:
#      30년간 납입 / 35년간 납입 / 40년간 납입
# 8 -> 5.1 / 4.7 / 4.3
# 6 -> 3 / 2.6 / 2.1
# 4 -> 1.3 / 0.9 / 0.5

# 0-year delayed

In [8]:
init_wealth = 350000
k = 0.5 # 고정

x = {
    10:[],
    15:[],
    20:[],
    25:[],
    30:[]
}
depletion = {
    10:[],
    15:[],
    20:[],
    25:[],
    30:[]
    
}
wealth = {
    10:[],
    15:[],
    20:[],
    25:[],
    30:[]
    
}

for upper_year in upper_years:
    print('upper_year:', upper_year)
    for withdraw_rate in withdraw_rates:
        for years in full_year[upper_year]:
            for year in years:
    #                 print('year:', year)
                if year in normal_year[upper_year][0]:
                    x[upper_year].append(0)
                elif year in bear_year[upper_year][0]:
                    x[upper_year].append(1)

                res = pv.get_metric(year, k, upper_year, withdraw_rate, df, init_wealth, p_val, w_val)
                if res[2] < upper_year*12 : #고갈 시점이 upper_year보다 작으면 (이미 고갈이 되면)
                    dep_or_not = 1
                elif res[2] >= upper_year*12: # 고갈이 안되면
                    dep_or_not = 0

                depletion[upper_year].append(dep_or_not)
                wealth[upper_year].append((res[1] + res[0])*350000/100/1000)

    print('dep prob:', depletion[upper_year])
    print('wealth:', wealth[upper_year])


upper_year: 20
dep prob: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
wealth: [460.27037461180515, 471.4112477984763, 477.9130325235957, 463.71784746892195, 497.98207041194246, 480.5515678715749, 509.74265953262227, 363.2773155186288, 266.7758255751191, 224.69497773164753, 231.03870522703906, 301.66900570543424, 298.9653893505266, 302.8858464674008, 264.646175925368, 273.4639449585207, 285.2037864279213, 335.9470485534613, 438.8082821581635, 427.98526933509044, 334.7674565647874, 475.5879077902128, 423.16427069706276, 437.3203772167587, 485.75439126142095, 485.2024345589307, 490.49596621144383, 509.6801986790166, 492.23106315596186, 499.6535370799796, 359.9980834151506, 280.3463998096905, 249.79583696529997, 234.28872420737372, 295.3929358115936, 295.3929358115936, 295.3929358115936, 265.14787859564694, 265.14787859564694, 280.3463998096905, 325.03611638894284, 409.7625854184395, 414.206845000

In [11]:
intercept_linear = {}
beta_linear = {}

intercept_logit = {}
beta_logit = {}
t_intercept_logit = {}
t_beta_logit = {}
r2 = {}

intercept_total = {}
beta_total = {}
t_intercept_total = {}
t_beta_total = {}
r2_total = {}

for upper_year in upper_years:
    print('upper_year:', upper_year)
    x_ = sm.add_constant(x[upper_year], prepend=False)
    mod = sm.OLS(depletion[upper_year], x_)#sm.OLS(depletion[upper_year], x_)
    res = mod.fit()
    print(res.summary())
#     print(res.prsquared)
    intercept_linear[upper_year] = np.round(res.params[1],3)
    beta_linear[upper_year] = np.round(res.params[0],3)
    
#     t_intercept[upper_year] = np.round(res.tvalues[1],3)
#     t_beta[upper_year] = np.round(res.tvalues[0],3)
    

for upper_year in upper_years:
    print('upper_year:', upper_year)
    x_ = sm.add_constant(x[upper_year], prepend=False)
    mod = sm.Logit(depletion[upper_year], x_)#sm.OLS(depletion[upper_year], x_)
    
    try:
    
        res = mod.fit()
        print(res.summary())
        intercept_logit[upper_year] = np.round(res.params[1],3)
        beta_logit[upper_year] = np.round(res.params[0],3)
    
        t_intercept_logit[upper_year] = np.round(res.tvalues[1],3)
        t_beta_logit[upper_year] = np.round(res.tvalues[0],3)
        r2[upper_year] = np.round(res.prsquared, 3)
        print(res.prsquared)
    except:
        intercept_logit[upper_year] = '-'
        beta_logit[upper_year] = '-'
    
        t_intercept_logit[upper_year] = '-'
        t_beta_logit[upper_year] = '-'
        r2[upper_year] = '-'

    
for upper_year in upper_years:
    print('upper_year:', upper_year)
    x_ = sm.add_constant(x[upper_year], prepend=False)
    mod = sm.OLS(wealth[upper_year], x_)
    res = mod.fit()
    print(res.summary())
#     print(res.prsquared)

    intercept_total[upper_year] = np.round(res.params[1],3)
    beta_total[upper_year] = np.round(res.params[0],3)
    
    t_intercept_total[upper_year] = np.round(res.tvalues[1],3)
    t_beta_total[upper_year] = np.round(res.tvalues[0],3)
    
    r2_total[upper_year] = np.round(res.rsquared_adj, 3)

upper_year: 20
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.148
Model:                            OLS   Adj. R-squared:                  0.129
Method:                 Least Squares   F-statistic:                     7.652
Date:                Mon, 10 Jul 2023   Prob (F-statistic):            0.00826
Time:                        19:36:48   Log-Likelihood:                -20.846
No. Observations:                  46   AIC:                             45.69
Df Residuals:                      44   BIC:                             49.35
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.3333      0.120     

/home/slcf/anaconda3/envs/hyungjinko/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [10]:
print('Intercept       & & {}   & {} & & {}   & & {}   & {} & & {}   & & {}   & {} & & {}'.format(intercept_logit[20], intercept_linear[20], intercept_total[20], intercept_logit[25], intercept_linear[25], intercept_total[25], intercept_logit[30], intercept_linear[30], intercept_total[30]))
print('                & & ({}) & -  & & ({}) & & ({}) & -  & & ({}) & & ({}) & -  & & ({})'.format(t_intercept_logit[20], t_intercept_total[20], t_intercept_logit[25], t_intercept_total[25], t_intercept_logit[30], t_intercept_total[30]))
print('Bearish (dummy) & & {}   & {} & & {}   & & {}   & {} & & {}   & & {}   & {} & & {}'.format(beta_logit[20], beta_linear[20], beta_total[20], beta_logit[25], beta_linear[25], beta_total[25], beta_logit[30], beta_linear[30], beta_total[30]))
print('                & & ({}) & -  & & ({}) & & ({}) & -  & & ({}) & & ({}) & -  & & ({})'.format(t_beta_logit[20], t_beta_total[20], t_beta_logit[25], t_beta_total[25], t_beta_logit[30], t_beta_total[30]))
print('Adj. $R^2$ & & \multicolumn2c{} & & {} & & \multicolumn2c{} & & {} & & \multicolumn2c{} & & {}'.format(r2[20], r2_total[20], r2[25], r2_total[25], r2[30], r2_total[30]))



Intercept       & & -34.858   & -0.0 & & 467.825   & & -2.708   & 0.063 & & 494.211   & & -1.946   & 0.125 & & 498.199
                & & (-0.0) &    & & (28.981) & & -2.622   &    & & (40.559) & & -2.574   &    & & (36.87)
Bearish (dummy) & & 34.165   & 0.333 & & -145.194   & & 3.114   & 0.538 & & -214.601   & & 3.332   & 0.675 & & -229.767
                & & (0.0) &    & & (-7.264) & & 2.757   &    & & (-13.127) & & 3.046   &    & & (-10.546)
Adj. $R^2$ & & \multicolumn2c0.207 & & 0.535 & & \multicolumn2c0.269 & & 0.83 & & \multicolumn2c0.363 & & 0.815


# 5-year delayed

In [7]:
upper_years = [15, 20, 25]
withdraw_rates = [0.026, 0.047] # 

In [8]:
init_wealth = 350000
k = 0.5 # 고정

x = {
    10:[],
    15:[],
    20:[],
    25:[]
}
depletion = {
    10:[],
    15:[],
    20:[],
    25:[]
    
}
wealth = {
    10:[],
    15:[],
    20:[],
    25:[]
    
}

for upper_year in upper_years:
    print('upper_year:', upper_year)
    for withdraw_rate in withdraw_rates:
        for years in full_year[upper_year]:
            for year in years:
                print('year:', year)
                if year in normal_year[upper_year][0]:
                    x[upper_year].append(0)
                elif year in bear_year[upper_year][0]:
                    x[upper_year].append(1)

                res = pv.get_metric(year, k, upper_year, withdraw_rate, df, init_wealth, p_val, w_val)
                if res[2] < upper_year*12 : #고갈 시점이 upper_year보다 작으면 (이미 고갈이 되면)
                    dep_or_not = 1
                elif res[2] >= upper_year*12: # 고갈이 안되면
                    dep_or_not = 0

                depletion[upper_year].append(dep_or_not)
                wealth[upper_year].append((res[1] + res[0])*350000/100/1000)

    print('dep prob:', depletion[upper_year])
    print('wealth:', wealth[upper_year])


upper_year: 15
year: 1980
year: 1981
year: 1982
year: 1983
year: 1984
year: 1985
year: 1986
year: 1987
year: 1988
year: 1989
year: 1990
year: 1991
year: 1992
year: 1993
year: 1994
year: 1995
year: 1996
year: 1997
year: 1998
year: 1999
year: 2000
year: 2001
year: 2002
year: 2003
year: 2004
year: 2005
year: 2006
year: 2007
year: 1980
year: 1981
year: 1982
year: 1983
year: 1984
year: 1985
year: 1986
year: 1987
year: 1988
year: 1989
year: 1990
year: 1991
year: 1992
year: 1993
year: 1994
year: 1995
year: 1996
year: 1997
year: 1998
year: 1999
year: 2000
year: 2001
year: 2002
year: 2003
year: 2004
year: 2005
year: 2006
year: 2007
dep prob: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
wealth: [536.2712903646222, 621.4148305694628, 539.5858252396265, 497.53718209640715, 481.4157342683795, 541.7894539242034, 428.5015748938273, 330.5004819691569, 228.3175903712084, 214.87413

In [9]:
intercept_linear = {}
beta_linear = {}

intercept_logit = {}
beta_logit = {}
t_intercept_logit = {}
t_beta_logit = {}
r2 = {}

intercept_total = {}
beta_total = {}
t_intercept_total = {}
t_beta_total = {}
r2_total = {}

for upper_year in upper_years:
    print('upper_year:', upper_year)
    x_ = sm.add_constant(x[upper_year], prepend=False)
    mod = sm.OLS(depletion[upper_year], x_)#sm.OLS(depletion[upper_year], x_)
    res = mod.fit()
    print(res.summary())
#     print(res.prsquared)
    intercept_linear[upper_year] = np.round(res.params[1],3)
    beta_linear[upper_year] = np.round(res.params[0],3)
    
#     t_intercept[upper_year] = np.round(res.tvalues[1],3)
#     t_beta[upper_year] = np.round(res.tvalues[0],3)
    

for upper_year in upper_years:
    print('upper_year:', upper_year)
    x_ = sm.add_constant(x[upper_year], prepend=False)
    mod = sm.Logit(depletion[upper_year], x_)#sm.OLS(depletion[upper_year], x_)
    
    try:
    
        res = mod.fit()
        print(res.summary())
        intercept_logit[upper_year] = np.round(res.params[1],3)
        beta_logit[upper_year] = np.round(res.params[0],3)
    
        t_intercept_logit[upper_year] = np.round(res.tvalues[1],3)
        t_beta_logit[upper_year] = np.round(res.tvalues[0],3)
        r2[upper_year] = np.round(res.prsquared, 3)
        print(res.prsquared)
    except:
        intercept_logit[upper_year] = '-'
        beta_logit[upper_year] = '-'
    
        t_intercept_logit[upper_year] = '-'
        t_beta_logit[upper_year] = '-'
        r2[upper_year] = '-'

    
for upper_year in upper_years:
    print('upper_year:', upper_year)
    x_ = sm.add_constant(x[upper_year], prepend=False)
    mod = sm.OLS(wealth[upper_year], x_)
    res = mod.fit()
    print(res.summary())
#     print(res.prsquared)

    intercept_total[upper_year] = np.round(res.params[1],3)
    beta_total[upper_year] = np.round(res.params[0],3)
    
    t_intercept_total[upper_year] = np.round(res.tvalues[1],3)
    t_beta_total[upper_year] = np.round(res.tvalues[0],3)
    
    r2_total[upper_year] = np.round(res.rsquared_adj, 3)

upper_year: 15
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                    0.6429
Date:                Mon, 10 Jul 2023   Prob (F-statistic):              0.426
Time:                        19:47:05   Log-Likelihood:                 34.085
No. Observations:                  56   AIC:                            -64.17
Df Residuals:                      54   BIC:                            -60.12
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0294      0.037     

/home/slcf/anaconda3/envs/hyungjinko/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/slcf/anaconda3/envs/hyungjinko/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [10]:
print('Intercept       & & {}   & {} & & {}   & & {}   & {} & & {}   & & {}   & {} & & {}'.format(intercept_logit[15], intercept_linear[15], intercept_total[15], intercept_logit[20], intercept_linear[20], intercept_total[20], intercept_logit[25], intercept_linear[25], intercept_total[25]))
print('                & & ({}) & -  & & ({}) & & ({}) & -  & & ({}) & & ({}) & -  & & ({})'.format(t_intercept_logit[15], t_intercept_total[15], t_intercept_logit[20], t_intercept_total[20], t_intercept_logit[25], t_intercept_total[25]))
print('Bearish (dummy) & & {}   & {} & & {}   & & {}   & {} & & {}   & & {}   & {} & & {}'.format(beta_logit[15], beta_linear[15], beta_total[15], beta_logit[20], beta_linear[20], beta_total[20], beta_logit[25], beta_linear[25], beta_total[25]))
print('                & & ({}) & -  & & ({}) & & ({}) & -  & & ({}) & & ({}) & -  & & ({})'.format(t_beta_logit[15], t_beta_total[15], t_beta_logit[20], t_beta_total[20], t_beta_logit[25], t_beta_total[25]))
print('Adj. $R^2$ & & \multicolumn2c{} & & {} & & \multicolumn2c{} & & {} & & \multicolumn2c{} & & {}'.format(r2[15], r2_total[15], r2[20], r2_total[20], r2[25], r2_total[25]))



Intercept       & & -12.71   & -0.0 & & 469.849   & & -34.862   & -0.0 & & 466.981   & & -2.708   & 0.063 & & 496.712
                & & (-0.104) & -  & & (27.182) & & (-0.0) & -  & & (28.323) & & (-2.622) & -  & & (39.192)
Bearish (dummy) & & 9.214   & 0.029 & & -140.291   & & 34.014   & 0.3 & & -143.051   & & 2.708   & 0.438 & & -216.567
                & & (0.075) & -  & & (-6.324) & & (0.0) & -  & & (-7.007) & & (2.406) & -  & & (-12.737)
Adj. $R^2$ & & \multicolumn2c0.101 & & 0.415 & & \multicolumn2c0.194 & & 0.517 & & \multicolumn2c0.206 & & 0.822


# 10

In [7]:
upper_years = [10, 15, 20]
withdraw_rates = [0.021, 0.043]

In [8]:
init_wealth = 350000
k = 0.5 # 고정

x = {
    10:[],
    15:[],
    20:[],
    25:[]
}
depletion = {
    10:[],
    15:[],
    20:[],
    25:[]
    
}
wealth = {
    10:[],
    15:[],
    20:[],
    25:[]
    
}

for upper_year in upper_years:
    print('upper_year:', upper_year)
    for withdraw_rate in withdraw_rates:

        for years in full_year[upper_year]:
            for year in years:
    #                 print('year:', year)
                if year in normal_year[upper_year][0]:
                    x[upper_year].append(0)
                elif year in bear_year[upper_year][0]:
                    x[upper_year].append(1)

                res = pv.get_metric(year, k, upper_year, withdraw_rate, df, init_wealth, p_val, w_val)
                if res[2] < upper_year*12 : #고갈 시점이 upper_year보다 작으면 (이미 고갈이 되면)
                    dep_or_not = 1
                elif res[2] >= upper_year*12: # 고갈이 안되면
                    dep_or_not = 0

                depletion[upper_year].append(dep_or_not)
                wealth[upper_year].append((res[1] + res[0])*350000/100/1000)


    print('upper_year:', upper_year)
    print('withdraw_rates:', withdraw_rates)
    print('dep prob:', depletion[upper_year])
    print('wealth:', wealth[upper_year])


upper_year: 10
upper_year: 10
withdraw_rates: [0.021, 0.043]
dep prob: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
wealth: [647.6132965948782, 629.814125386532, 607.2765821376, 638.2731304911547, 698.1456915181158, 640.2373204309292, 581.5868497168093, 372.0144402245938, 237.06796623434963, 206.25332754590295, 243.58501441107921, 253.88163290440792, 267.6177981651642, 242.47765038612988, 238.07512080930655, 248.87093542907422, 297.4986859603378, 342.4771177708642, 441.84566159842956, 380.6875143980735, 334.7937798088071, 471.51528669225536, 420.2657490494749, 487.86046767542854, 417.1912933704067, 408.6537838954789, 337.92405122051133, 349.66060668766517, 357.18243045742855, 410.1687792005633, 353.87160446973473, 362.8621706951067, 358.2817861195766, 575.6278429708393, 594.6926190967408, 580.0737296998109, 617.9850878269605, 673.3535

In [9]:
intercept_linear = {}
beta_linear = {}

intercept_logit = {}
beta_logit = {}
t_intercept_logit = {}
t_beta_logit = {}
r2 = {}

intercept_total = {}
beta_total = {}
t_intercept_total = {}
t_beta_total = {}
r2_total = {}

for upper_year in upper_years:
    print('upper_year:', upper_year)
    x_ = sm.add_constant(x[upper_year], prepend=False)
    mod = sm.OLS(depletion[upper_year], x_)#sm.OLS(depletion[upper_year], x_)
    res = mod.fit()
    print(res.summary())
#     print(res.prsquared)
    intercept_linear[upper_year] = np.round(res.params[1],3)
    beta_linear[upper_year] = np.round(res.params[0],3)
    
#     t_intercept[upper_year] = np.round(res.tvalues[1],3)
#     t_beta[upper_year] = np.round(res.tvalues[0],3)
    

for upper_year in upper_years:
    print('upper_year:', upper_year)
    x_ = sm.add_constant(x[upper_year], prepend=False)
    mod = sm.Logit(depletion[upper_year], x_)#sm.OLS(depletion[upper_year], x_)
    
    try:
    
        res = mod.fit()
        print(res.summary())
        intercept_logit[upper_year] = np.round(res.params[1],3)
        beta_logit[upper_year] = np.round(res.params[0],3)
    
        t_intercept_logit[upper_year] = np.round(res.tvalues[1],3)
        t_beta_logit[upper_year] = np.round(res.tvalues[0],3)
        r2[upper_year] = np.round(res.prsquared, 3)
        print(res.prsquared)
    except:
        intercept_logit[upper_year] = '-'
        beta_logit[upper_year] = '-'
    
        t_intercept_logit[upper_year] = '-'
        t_beta_logit[upper_year] = '-'
        r2[upper_year] = '-'

    
for upper_year in upper_years:
    print('upper_year:', upper_year)
    x_ = sm.add_constant(x[upper_year], prepend=False)
    mod = sm.OLS(wealth[upper_year], x_)
    res = mod.fit()
    print(res.summary())
#     print(res.prsquared)

    intercept_total[upper_year] = np.round(res.params[1],3)
    beta_total[upper_year] = np.round(res.params[0],3)
    
    t_intercept_total[upper_year] = np.round(res.tvalues[1],3)
    t_beta_total[upper_year] = np.round(res.tvalues[0],3)
    
    r2_total[upper_year] = np.round(res.rsquared_adj, 3)

upper_year: 10
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                         nan
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Mon, 10 Jul 2023   Prob (F-statistic):                nan
Time:                        19:51:29   Log-Likelihood:                    inf
No. Observations:                  66   AIC:                              -inf
Df Residuals:                      64   BIC:                              -inf
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1                  0          0     

/home/slcf/anaconda3/envs/hyungjinko/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/home/slcf/anaconda3/envs/hyungjinko/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1804: RuntimeWarning: invalid value encountered in double_scalars
  return self.mse_model/self.mse_resid
/home/slcf/anaconda3/envs/hyungjinko/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:903: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/home/slcf/anaconda3/envs/hyungjinko/lib/python3.8/site-packages/statsmodels/stats/stattools.py:50: RuntimeWarning: invalid value encountered in double_scalars
  dw = np.sum(diff_resids**2, axis=axis) / np.sum(resids**2, axis=axis)
/home/slcf/anaconda3/envs/hyungjinko/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning

In [10]:
print('Intercept       & & {}   & {} & & {}   & & {}   & {} & & {}   & & {}   & {} & & {}'.format(intercept_logit[10], intercept_linear[10], intercept_total[10], intercept_logit[15], intercept_linear[15], intercept_total[15], intercept_logit[20], intercept_linear[20], intercept_total[20]))
print('                & & ({}) & -  & & ({}) & & ({}) & -  & & ({}) & & ({}) & -  & & ({})'.format(t_intercept_logit[10], t_intercept_total[10], t_intercept_logit[15], t_intercept_total[15], t_intercept_logit[20], t_intercept_total[20]))
print('Bearish (dummy) & & {}   & {} & & {}   & & {}   & {} & & {}   & & {}   & {} & & {}'.format(beta_logit[10], beta_linear[10], beta_total[10], beta_logit[15], beta_linear[15], beta_total[15], beta_logit[20], beta_linear[20], beta_total[20]))
print('                & & ({}) & -  & & ({}) & & ({}) & -  & & ({}) & & ({}) & -  & & ({})'.format(t_beta_logit[10], t_beta_total[10], t_beta_logit[15], t_beta_total[15], t_beta_logit[20], t_beta_total[20]))
print('Adj. $R^2$ & & \multicolumn2c{} & & {} & & \multicolumn2c{} & & {} & & \multicolumn2c{} & & {}'.format(r2[10], r2_total[10], r2[15], r2_total[15], r2[20], r2_total[20]))



Intercept       & & -   & 0.0 & & 496.909   & & -   & 0.0 & & 470.439   & & -34.856   & -0.0 & & 466.031
                & & (-) & -  & & (24.934) & & (-) & -  & & (26.6) & & (-0.0) & -  & & (27.602)
Bearish (dummy) & & -   & 0.0 & & -169.125   & & -   & 0.0 & & -139.671   & & 33.247   & 0.167 & & -140.751
                & & (-) & -  & & (-6.439) & & (-) & -  & & (-6.154) & & (0.0) & -  & & (-6.732)
Adj. $R^2$ & & \multicolumn2c- & & 0.384 & & \multicolumn2c- & & 0.401 & & \multicolumn2c0.145 & & 0.496
